In [ ]:
# Install required libraries
!pip install transformers datasets evaluate accelerate peft trl bitsandbytes
!pip install seaborn matplotlib scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Finalized LoRA-based RoBERTa for AGNEWS (Optimized for ~83%+ accuracy with noise filtering + manual augmentation)
import torch
import os
os.environ["WANDB_DISABLED"]="true"
import torch.nn as nn
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import (
    RobertaForSequenceClassification, RobertaTokenizer,
    Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, classification_report
import re
import random

# Set seed and device
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
dataset = load_dataset("ag_news")
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Clean text: remove special characters, excessive whitespace
def clean_text(example):
    text = example['text']
    text = re.sub(r"[^\w\s.,!?]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    example['text'] = text
    return example

dataset = dataset.map(clean_text)

# Remove short texts (< 8 words)
def filter_short(example):
    return len(example['text'].split()) >= 8

dataset = dataset.filter(filter_short)

# Remove duplicates
unique_texts = set()
def deduplicate(example):
    text = example['text']
    if text not in unique_texts:
        unique_texts.add(text)
        return True
    return False

dataset = dataset.filter(deduplicate)

# Manual augmentation: shuffle internal words for basic rephrasing
def augment_text(text):
    words = text.split()
    if len(words) > 8:
        mid = words[1:-1]
        random.shuffle(mid)
        return " ".join([words[0]] + mid + [words[-1]])
    return text

# Apply manual EDA to 20% of training set
df_train = pd.DataFrame(dataset['train'])
sample_size = int(0.2 * len(df_train))
aug_rows = df_train.sample(n=sample_size, random_state=SEED)
aug_texts = [augment_text(t) for t in aug_rows['text']]
aug_labels = aug_rows['label'].tolist()

aug_df = pd.DataFrame({"text": aug_texts, "label": aug_labels})
full_df = pd.concat([df_train, aug_df], ignore_index=True)

from datasets import Dataset
train_dataset_raw = Dataset.from_pandas(full_df)
dataset['train'] = train_dataset_raw

# Tokenization
def preprocess(example):
    return tokenizer(example['text'], truncation=True, padding='max_length', max_length=192)

tokenized = dataset.map(preprocess, batched=True)
tokenized = tokenized.rename_column("label", "labels")
tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Split
split = tokenized["train"].train_test_split(test_size=0.1, seed=SEED)
train_dataset = split["train"]
eval_dataset = split["test"]
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Load base model
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForSequenceClassification

base_model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=4)

# LoRA config
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=[
        "roberta.encoder.layer.9.attention.self.query",
        "roberta.encoder.layer.10.attention.self.query",
        "roberta.encoder.layer.11.attention.self.query",
        "roberta.encoder.layer.11.output.dense"
    ]
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

# Compute metrics
label_names = ["World", "Sports", "Business", "Sci/Tech"]

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    report = classification_report(labels, preds, output_dict=True, target_names=label_names)
    results = {"accuracy": acc}
    for cname in label_names:
        results[f"{cname}_f1"] = report[cname]['f1-score']
    return results

# Trainer setup
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        weights = torch.ones(4, device=logits.device)
        weights[2] = 1.25  # Boost business
        loss_fct = nn.CrossEntropyLoss(weight=weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

training_args = TrainingArguments(
    output_dir="lora_agnews_final",
    num_train_epochs=15,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    learning_rate=3e-4,
    weight_decay=0.02,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    logging_dir="./logs",
    seed=SEED
)

trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()

# Final eval
metrics = trainer.evaluate()
print(f"\nFinal Test Accuracy: {metrics['eval_accuracy']:.4f}")

Filter:   0%|          | 0/7600 [00:00<?, ? examples/s]

Map:   0%|          | 0/143887 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-16-651c92b31caf>:157: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedLossTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedLossTraine

trainable params: 728,836 || all params: 125,377,544 || trainable%: 0.5813


No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,World F1,Sports F1,Business F1,Sci/tech F1
1,0.276800,0.243736,0.922163,0.929069,0.974632,0.887124,0.896861
2,0.232000,0.227322,0.926541,0.935304,0.979659,0.890764,0.899400
3,0.246200,0.215158,0.931475,0.939487,0.979981,0.900396,0.905552
4,0.218800,0.198430,0.933560,0.942206,0.980764,0.903884,0.906495
5,0.204700,0.210231,0.933491,0.944483,0.984153,0.900338,0.905371
6,0.205600,0.195572,0.936132,0.947486,0.983018,0.903670,0.909869
7,0.200800,0.187992,0.938217,0.946191,0.985467,0.907000,0.914018
8,0.195000,0.185705,0.938703,0.946148,0.985908,0.910899,0.911969
9,0.181300,0.189534,0.940302,0.949818,0.986981,0.909649,0.914774
10,0.179500,0.182962,0.941205,0.952249,0.986840,0.911589,0.913733



Final Test Accuracy: 0.9434


In [ ]:
import torch
import pandas as pd
import re
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import RobertaTokenizer
from datasets import Dataset

# Load tokenizer and model (already trained LoRA-based model assumed)
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model.eval()
model.to("cuda" if torch.cuda.is_available() else "cpu")

# Load test_unlabelled.pkl
test_dataset = pd.read_pickle("/content/test_unlabelled.pkl")  # Or wherever your file is

# If it's HuggingFace Dataset, convert to pandas
if not isinstance(test_dataset, pd.DataFrame):
    test_dataset = test_dataset.to_pandas()

# Clean text
def clean_text(text):
    text = re.sub(r"[^\w\s.,!?]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

test_dataset['text'] = test_dataset['text'].apply(clean_text)

# Convert back to HuggingFace Dataset
test_dataset_hf = Dataset.from_pandas(test_dataset)

# Tokenize
def preprocess(example):
    return tokenizer(example['text'], truncation=True, padding='max_length', max_length=192)

test_dataset_hf = test_dataset_hf.map(preprocess, batched=True)

# Set tensor format
test_dataset_hf.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Create DataLoader
test_loader = DataLoader(test_dataset_hf, batch_size=32)

# Predict
all_preds = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=-1)
        all_preds.extend(preds.cpu().numpy())

# Create submission DataFrame
submission = pd.DataFrame({
    "ID": range(len(all_preds)),
    "Label": all_preds
})

# Save to CSV
submission.to_csv("/content/Submission.csv", index=False)
print("✅ submission.csv saved!")


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

100%|██████████| 250/250 [00:17<00:00, 14.40it/s]

✅ submission.csv saved!
